In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen 
import re
from bancor3_simulator.protocol import BancorV3
from bancor3_simulator.core.settings import GlobalSettings
from decimal import Decimal
import pandas as pd

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

# Rewards Mechanisms

The liquidity mining program in Bancor V2.1 is a gas intensive operation, and an expensive activity for many liquidity providers. The single greatest burden in its implementation is the multiplier logic, which queries user position data. As user positions will no longer exist in Bancor V3, this specific feature will cease to operate. The multiplier logic is therefore forfeited, alleviating the single greatest source of friction for users vis-a-vis the gas costs of interacting with the Bancor Network. However, there are further optimizations that allow for simplification not just on the contract level, but also the demand for user erudition.

In V2.1, BNT staking rewards can be 'claimed', which sends BNT to the user's wallet address, or 'restaked', which adds BNT to a liquidity pool and creates a new position on the user's behalf. The distinction between these operations was important for determining the behavior of the multiplier, the absence of which warrants a close examination of whether these two separate operations are still meaningful.

The process of staking rewards is relatively simple to understand. For the purpose of demonstration, assume Alice has accrued 100 BNT in liquidity mining rewards, and wishes to restake them. Until she takes an action, these BNT are entirely hypothetical; they do not exist anywhere, and the rewards number is closer to a 'BNT minting permission' than anything else. When Alice commits to the staking action, she sends a request to the protocol for the rewards to be minted, and elects the pool to which the rewards should be deposited. The protocol then mints the BNT, and deposits it to the pool chosen by Alice.

This procedure contains a large number of completely redundant operations. The redundancy becomes visible when consideration is paid to how the protocol handles BNT deposits. To maintain the current price point in any pool, the addition of BNT to any reserve must be commensurate with the withdrawal of an equal amount of protocol-owned BNT, which is subsequently destroyed. Therefore, the protocol simultaneously mints 100 BNT, and burns 100 BNT, which is a zero-sum operation. In Bancor V2.1, these steps are in fact necessary, as the deposit and withdrawal actions affect the unique user positions held by Alice and the protocol. However, the high-level consequences are the same; the protocol lost 100 BNT and Alice gained 100 BNT. Therefore, the protocol transfers its own BNT to Alice, and the minting/burning steps can be ignored.

![](https://lh4.googleusercontent.com/1k7A8xllBQFPrjB7BFJ6yXL6zbjWBBv2H1lOMrUjIvQgBQlMf3CrFQ74XopNWSXw9pCgHl0zJmO3nJmdvJ0R6U9kov5-S1hVIiPZiQpQfzFp_mcTIywS4JjjtRTFGlzl2V4KVXu8)

The removal of user positions, and the transferability of bnBNT pool tokens in Bancor V3 offer a drastic simplification of the above process. The bnBNT pool token value is determined by their number, relative to the total staked balance of BNT. Therefore, the protocol can relinquish ownership of its BNT to users, simply by destroying its bnBNT. There are no transfers, and no minting events. The burn action is attached to the Vortex trigger, meaning that the rewards are compounded every time the vortex burner performs its action. These duties will be partially supported through integration with the Chainlink Keepers network.

![](https://lh5.googleusercontent.com/x3ozVIXauD-wduD98MWMV9XySwK8cYhV8Dghf7IJDAkzIar1dpoTQrzFJnrF_gtrZy4g8UiXF7PVIkN0WeUAiwUyMVIWHsxzcckW__6lNKJ_owGiz6LBUtz-IDSFiLd2rjmixLEx)

The distinction between 'staking' and 'claiming' becomes less important under this system. The restaking action is essentially performed by default. As Alice has the freedom to liquidate any number of her bnBNT pool tokens at any time, she also retains the ability to 'claim' rewards as she sees fit. If any distinction remains, it is that the act of 'claiming rewards' requires an action, whereas 'restaking rewards' requires no input from the user whatsoever.

Consider the following simplified case. Alice and the protocol have 20 bnBNT and 80 bnBNT, respectively, representing a share of a total of 100 BNT. At a future date, the protocol destroys 9.1 bnBNT, causing Alice's share value to increase by 10%. Whereas Alice originally had access to only 20 BNT, she can now withdraw up to 22 BNT. Claiming rewards is akin to withdrawing interest-only, while leaving the principle locked. To achieve this, Alice simply liquidates 1.82 bnBNT for 2 BNT. It is noteworthy that under this system with access only to the bnBNT token value, speaking entirely from the user's perspective, the APY from liquidity mining will be indistinguishable from other sources of value accrual; there is an opportunity to display a single APY, removing a common source of confusion on the user interface.

![](https://lh6.googleusercontent.com/fCtg-_RFluPfoAYygfux9fR2WViskD35DJGhpBHvvMurxBK8DrG9ZB9p9MKv98jxdF0mdIEUGOGVbPjJzdX_vtMzaMRSDxFNxHIw0xDsLCYmAF4gIPSf3jvXv8EEEKJ1QMl5bxll)

Rewards Schedules 
------------------

The rewards budget for Bancor 3 is subject to its own proposal; the specific settings described here are for the sake of demonstration, only. The final settings are detailed in BIP 16, and subject to community approval via the governance process.

The new staking rewards system is designed to promote a sustained BNT ecosystem, with a moderate inflation rate over a longer time period than was envisioned for Bancor v2.1. In V3, a maximum of 40,000,000 BNT will be distributed over an infinite time period. The distribution rate follows an exponential decay curve.

![](https://lh5.googleusercontent.com/_RBrufqZ0mwxhh5A6dSUDtV3tYwk_H7abZwlpoRii0c2j1BWSBlawrCahkMmE_yq4_c8ztBtHVMgjy_VbzhesnBcv_oQ2ZsSMhfJ0lcXrFgFB37BJIW-yaEP9m6oNT38621uq_xN)

Where N0 is the total rewards that will ever be distributed, and Nt are the remaining rewards to be distributed at any block number, t. The total rewards issued upto and including any block number, N-t, is given by:

![](https://lh6.googleusercontent.com/d_bg28FxJllgWAR4ebHqba57j45LIixJMAwCGosxfuVgokkNrWtjXGwT9LY3r2UE5Xla2-mrMEITazqGJuf277S_R7PGaG1SJvba8ecClVfQr7UK9fppEU_QrDCYAbMJ52VxtGYU)

The decay constant, λ, is a number chosen so as to achieve a desired inflation profile and should be considered in units of block-1, and quoted in ppm format. The commencement block, t0, is the block number in which the staking rewards contract is deployed (or close to it). There are only two variables that dictate the behavior of the rewards program, N0 and λ. The total number of BNT to be distributed over infinite time is determined by N0. The specific values chosen for these variables have profound implications for the apparent APY, and inflation rate of BNT.

For the purpose of this document, the 'true' BNT supply is taken as 160,000,000 as of 06 July 2021. Deploying the rewards contract with values for N0 and λ of 40,000,000 and 0.2 ppm, respectively, targets an approximate cumulative inflation rate of 9% after 1 year, 15% after two years, 19% after three years, 21% after four years, 22% after five years, and approaches a maximum of 25% total inflation after infinite time. These figures are approximate, and have been calculated assuming an average of 6500 blocks per day on Ethereum Layer One. The distribution rate begins at 8 BNT per block, and slowly approaches zero thereafter.

![](https://lh4.googleusercontent.com/eRBGc8n-Gq8GlSb_n-1nNuajcmNa9HokiTRpzErFaYsFwt3zEwgr3UJSRLbeOPQr6EDSiJpaJ-GNFl7haPksz5iyzrO31aMgjGh2jrBbuyT3VHvXnMBh3oYfKYO97j2y7S6qvqw-)

The calculation affecting the amount of bnBNT to burn is not as simple as it might first appear. The following expression describes an accurate transfer  of a number of BNT out of the protocol's possession by adjusting its share of the total bnBNT:

![](https://lh3.googleusercontent.com/o5E9yyZCwZo8RyIIitikRtLIRztwcLBchgcRb1_swq9bi_O93K1ZhIDDkWvB3KI1rpvf1ztkjGp2XT-mH9VaWVyC0KHAlP99ip62K7zK6Xxm_9vPVTlKJvUOnkO1NnVc71LgHWbr)

where x is the amount of bnBNT pool tokens to be burned, a is the total amount of BNT in the staking ledger, b is the amount of BNT to distribute, c is the total supply of bnBNT, and d is the amount of bnBNT held by the protocol.

The initiation of a rewards calculation should piggyback on another operation. Further burdening liquidity providers with excess gas expenditure can be avoided by leveraging the incentives already built into the vortex contract. The vortex is a routine maintenance operation, which external participants routinely activate in return for a small reward. The rewards distribution calculation can be dovetailed into the same incentives program, ensuring that the calculations and rewards distribution is performed regularly.

The method is uncomplicated. In addition to the constants t0, N0 and λ, each time a rewards distribution occurs, the contract need only know:

-   the most recent confirmed block number, 

-   the value of N-t recorded during the last rewards distribution,

-   the total supply of bnBNT,

-   the number of protocol-owned bnBNT, and

-   the total BNT staked in the protocol (both protocol- and user-owned).

To illustrate the process, assume the rewards contract was deployed on block 12798451 (therefore becoming the constant t0). Then, approximately 15 minutes later on block 12798517, the vortex burn action is triggered by a user. This block becomes the variable t, and the number of rewards that should have been distributed up to this point are calculated (N-t). This number, 528.00 (b), is stored within the contract, and is used in much the same way as a running total. The last BNT rewards distribution amount is subtracted from this number to give the total BNT amount to be distributed at this moment in time. In this example there have been no prior BNT rewards, so the amount subtracted is 0 and the BNT to be distributed on this block is 528.00.

For this example assume that the amount of BNT staked in the system, a, is 10,000,000. The total supply of bnBNT, c, is 1,000,000, of which 500,000 is protocol-owned (d). These numbers and the amount of BNT rewards to distribute (b), when substituted into the equation above, return the number of bnBNT pool tokens to burn (x). In this scenario, the procol burns 105.59 bnBNT to distribute 528.00 BNT to users.

Now assume a second vortex burn occurs another 20 minutes later, on block 12798617, which becomes the new variable t, and the new running total (N-t) is 1,327.98. The last entry, 528.00, is subtracted from this number to give 799.98, which is the amount of BNT rewards to be distributed on this block (b). Note that the values for c and d have changed, as the burning of bnBNT has affected both the total bnBNT supply and the protocol-held bnBNT; however, the total BNT staked in the system remains the same. Substituting these values into the same equation as before returns the value 159.94 - the number of bnBNT the protocol will burn on this block to distribute 799.98 BNT to users.

![](https://lh6.googleusercontent.com/I6bYiGBMxtlkhinV_IHBh6nkLCIPNIRRttBnc8A4e2D2CMZktUKW3Cp6qs-7XfQsvT1EJfEBXzi7ruQmljjP2GZg7t7TJJzL7tAzkDf-pKUMRB4CGxKMB3S-Bnyv0QSSzaxwCpp3)

This process continues literally forever. The decay is such that the total emissions rate will be reduced by half approximately every 1.5 years. During the first 76 weeks of operation, this reward schedule will distribute approximately 20,000,000 BNT; over the following 76 weeks 10,000,000 BNT will be distributed, and so on, until the end of time.